# SarvUday v2
-  Fine Tuning Base Model of SurvUday (survuday_v1): `/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v1`
- The Base Model was developed by Fine Tuning Llama 3.2 (3B)
- Dataset Description: Trained on 30,000 data samples including both mental health related and non-mental health related. Around 85% data related to mental health and 15% data related to non-mental health
- Dataset -> `mental_health_corpus_02`
- Dataset Link -> `/content/drive/MyDrive/Projects/Data/mental_health_corpus_02.csv`

### Install

In [ ]:
%%capture
import os
from google.colab import drive
drive.mount('/content/drive')

os.environ["WANDB_PROJECT"] = "SarvUday_v2"
os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_API_KEY"] = ""

!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install wandb

import random
import pandas as pd
from datasets import load_dataset , Dataset
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template

### Train

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v1",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

output_dir = "/content/drive/MyDrive/Projects/llama_3_2_3B_chat_v2"
data_path = '/content/drive/MyDrive/Projects/Data/mental_health_corpus_02.csv'
df = pd.read_csv(data_path)
df.head()

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

,conversations
0,"[{'from': 'human', 'value': ""Hi SurvUday, I ho..."
1,"[{'from': 'human', 'value': ""I've been feeling..."
2,"[{'from': 'human', 'value': ""Hi SurvUday, I ho..."
3,"[{'from': 'human', 'value': ""I've been feeling..."
4,"[{'from': 'human', 'value': 'I seek peace with..."


In [ ]:
import ast

df = df.dropna(subset=['conversations'])
# Convert string literals to lists only for non-null values
df['conversations'] = df['conversations'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Define tokenizer and other setup code
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True,
)

# Function to apply formatting prompts
def formatting_prompts_func(examples):
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in examples["conversations"]]
    return {"text": texts}

dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/29959 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': "Hi SurvUday, I've been feeling really stressed lately because I'm experiencing conflict with some of my friends. It's been taking a toll on me, and I'm not sure how to deal with it. I want to find a way to resolve these conflicts and find success in my relationships again. "},
 {'from': 'gpt',
  'value': "Hi, thank you for sharing that with me. It takes a lot of courage to open up about the challenges we face in our friendships. I can see that you're feeling stressed, and it's important to address these conflicts in a healthy way. Can you tell me a bit more about what's been happening and how it's been affecting you? "},
 {'from': 'human',
  'value': "Well, it all started when my friends and I disagreed on a certain issue. It escalated quickly, and now we're not speaking to each other. This conflict has left me feeling isolated and hurt. I value these friendships, and I want to find a way to reconcile and move forward together. "},
 {'from': 'gpt',
  'val

In [ ]:
print(dataset[5]["text"])

<|im_start|>user
Hi SurvUday, I've been feeling really stressed lately because I'm experiencing conflict with some of my friends. It's been taking a toll on me, and I'm not sure how to deal with it. I want to find a way to resolve these conflicts and find success in my relationships again. <|im_end|>
<|im_start|>assistant
Hi, thank you for sharing that with me. It takes a lot of courage to open up about the challenges we face in our friendships. I can see that you're feeling stressed, and it's important to address these conflicts in a healthy way. Can you tell me a bit more about what's been happening and how it's been affecting you? <|im_end|>
<|im_start|>user
Well, it all started when my friends and I disagreed on a certain issue. It escalated quickly, and now we're not speaking to each other. This conflict has left me feeling isolated and hurt. I value these friendships, and I want to find a way to reconcile and move forward together. <|im_end|>
<|im_start|>assistant
I can imagine h

In [ ]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,),
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
        map_eos_token = True,
    )

In [ ]:
exist_already = os.path.exists(output_dir)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        save_strategy = "steps",
        save_steps = 5,
        # eval_steps= 100,
        save_total_limit = 2,
        report_to="wandb"
    ),
)

if(exist_already):
  trainer_stats = trainer.train(resume_from_checkpoint=True)
else:
  trainer_stats = trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss
711,0.770400
712,0.701000
713,0.761500
714,0.738100
715,0.730500
716,0.729100
717,0.735200
718,0.688000
719,0.690300
720,0.733700


### Inference

In [ ]:
def generate_response(max_new_tokens=512):
    # Ask for user input
    user_input = input("Enter your message: ")

    messages = [
        {"from": "human", "value": f"{user_input}"},
    ]

    # Format and tokenize the input
    FastLanguageModel.for_inference(model)
    # Format the input message with tokenizer
    tokenized_input = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Since tokenized_input is a tensor, we directly pass it to model.generate
    inputs = {"input_ids": tokenized_input}

    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=max_new_tokens, use_cache=True)

In [ ]:
generate_response()

Enter your message: I am feeling low, what should I do now?
<|im_start|>user
I am feeling low, what should I do now?<|im_end|>
<|im_start|>assistant
Hi, I'm here to help. I can sense that you're feeling down. Can you tell me more about what's been going on? <|im_end|>


### Save Lora Adapters

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v2")
tokenizer.save_pretrained("/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v2")

('/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v2/tokenizer_config.json',
 '/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v2/special_tokens_map.json',
 '/content/drive/MyDrive/Projects/models/llama_3_2_3B_SurvUday_v2/tokenizer.json')

### Save Q4_K_M

In [ ]:
Token = "" # Huggingface Token
Repo = "dkp2701/survuday_v2" # edit
model.push_to_hub_gguf(Repo,
                        tokenizer,
                        quantization_method = ["q4_k_m"],
                        token = Token)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.92 out of 12.67 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 16.99it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving dkp2701/survuday_v2/pytorch_model-00001-of-00002.bin...
Unsloth: Saving dkp2701/survuday_v2/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at dkp2701/survuday_v2 into f16 GGUF format.
The output location will be /content/dkp2701/survuday_v2/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: survuday_v2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:ro

unsloth.Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/dkp2701/survuday_v2


Saved Ollama Modelfile to https://huggingface.co/dkp2701/survuday_v2
